In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [4]:
train_dir = './dogs-vs-cats/train'
test_dir = './dogs-vs-cats/test'

In [5]:
import glob

train_list = glob.glob(os.path.join(train_dir,'*.jpg'))
test_list = glob.glob(os.path.join(test_dir, '*.jpg'))

In [6]:
from PIL import Image

In [7]:
from sklearn.model_selection import train_test_split
train_list, val_list = train_test_split(train_list, test_size=0.2)

In [8]:
train_transforms =  transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])

val_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])


test_transforms = transforms.Compose([   
    transforms.Resize((224, 224)),
     transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
    ])

In [9]:
class dataset(torch.utils.data.Dataset):
    def __init__(self,file_list,transform=None):
        self.file_list = file_list
        self.transform = transform
        
        
    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength
    
    def __getitem__(self,idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        
        label = img_path.split('/')[-1].split('.')[0].split('\\')[-1]
        if label == "dog":
            label = 1
        else:
            label = 0
        
        return img_transformed,label

In [10]:
train_data = dataset(train_list, transform=train_transforms)
test_data = dataset(test_list, transform=test_transforms)
val_data = dataset(val_list, transform=test_transforms)

In [11]:
batch_size = 100
epoch = 10
lr = 0.001

In [12]:
train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size=batch_size, shuffle=True )
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset = val_data, batch_size=batch_size, shuffle=True)

In [13]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = torch.nn.Conv2d(3,10,kernel_size = 5)
        self.conv2 = torch.nn.Conv2d(10,20,kernel_size = 5)
        self.pooling = torch.nn.MaxPool2d(2)
        self.fc1 = torch.nn.Linear(56180,1000)
        self.fc2 = torch.nn.Linear(1000,2)
        
    def forward(self,x):
        batch_size = x.size(0)
#         x为张量，张量.size 取出维度  取0  得到就是样本数量 n 3 224 224
        x = x.view(batch_size,3,224,224)
        x = self.pooling(F.relu(self.conv1(x)))
        x = self.pooling(F.relu(self.conv2(x)))
        x = x.view(batch_size,-1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [14]:
model = Net()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01,momentum=0.5)

In [15]:
def train_func(epoch):
    running_loss = 0.0
    for batch_idx,data in enumerate(train_loader,0):
        inputs,target = data
#         inputs,target = inputs.to(device),target.to(device)
        optimizer.zero_grad()
        
        
#         forward and backward and update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 100 == 99:
            print('[%d,%5d] loss: %.3f' % (epoch + 1,batch_idx + 1,running_loss / 100))
            running_loss =0.0

In [16]:
def test_func():
    correct = 0
    total = 0
    with torch.no_grad():
#         执行之后在下面代码就不会执行梯度
        for data in val_loader:
            images,labels = data
#             images,labels = images.to(device),labels.to(device)
#             拿数据
            outputs = model(images)
#             做预测，拿到的结果是一个矩阵，每一行都是一个独热向量
            _, predicted = torch.max(outputs.data,dim = 1)
#           返回 最大值 和 每一行的最大值下标
#           指定沿着维度1（往下 行是第0个维度，向右 列是第一个维度）
            total += labels.size(0)
#             label是一个N 1元组 size 取 0 就是？

            correct += (predicted == labels).sum().item()
    print('Accuracy on test set: %d %%' % (100 * correct / total))

In [17]:
train_func(2)

[3,  100] loss: 0.693
[3,  200] loss: 0.690


In [26]:
def predict_func():
    dog_probs=[]
    with torch.no_grad():
        for data,fileid in test_loader:
            preds = model(data)
            preds_list = F.softmax(preds,dim=1)[:, 1].tolist()
            dog_probs += list(zip(list(fileid),preds_list))
    dog_probs.sort(key = lambda x : int(x[0]))
    idx = list(map(lambda x: x[0],dog_probs))
    prob = list(map(lambda x: x[1],dog_probs))
    submission = pd.DataFrame({'id':int(idx),'label':prob})
    submission.to_csv('result.csv',index=False)

In [27]:
predict_func()

In [28]:
if __name__ == '__main__':
    for epoch in range(19):
        train_func(epoch)
        test_func()
    predict_func()

[1,  100] loss: 0.687
[1,  200] loss: 0.685
Accuracy on test set: 56 %
[2,  100] loss: 0.681
[2,  200] loss: 0.678
Accuracy on test set: 57 %
[3,  100] loss: 0.675
[3,  200] loss: 0.668
Accuracy on test set: 58 %
[4,  100] loss: 0.667
[4,  200] loss: 0.660
Accuracy on test set: 60 %
[5,  100] loss: 0.658
[5,  200] loss: 0.655
Accuracy on test set: 61 %
[6,  100] loss: 0.647
[6,  200] loss: 0.645
Accuracy on test set: 63 %
[7,  100] loss: 0.637
[7,  200] loss: 0.633
Accuracy on test set: 63 %
[8,  100] loss: 0.631
[8,  200] loss: 0.623
Accuracy on test set: 64 %
[9,  100] loss: 0.623
[9,  200] loss: 0.610
Accuracy on test set: 65 %
[10,  100] loss: 0.609
[10,  200] loss: 0.610
Accuracy on test set: 66 %
[11,  100] loss: 0.610
[11,  200] loss: 0.612
Accuracy on test set: 66 %
[12,  100] loss: 0.601
[12,  200] loss: 0.594
Accuracy on test set: 67 %
[13,  100] loss: 0.600
[13,  200] loss: 0.595
Accuracy on test set: 67 %
[14,  100] loss: 0.595
[14,  200] loss: 0.593
Accuracy on test set: 6

In [29]:
!ls

'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
